In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso,Ridge, ElasticNet, LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

In [ ]:
#recuperamos la salida del proceso04
departamentos = pd.read_csv('../data/out_proceso04.csv', decimal=".")

departamentos.fillna(0, inplace=True)

departamentos.columns

In [ ]:
#eliminamos variables que no vamos a usar en el modelo
departamentos.drop(['operacion', 'tipo_propiedad', 'pais', 'provincia', 'ciudad_barrio','emprendimiento',
                    'barrio', 'barrio_cerrado', 'geonames_url', 'precio', 'moneda', 'titulo', 'descripcion',
                    'descripciontrip','geometry'], axis=1, inplace=True)

In [ ]:
departamentos.columns

In [ ]:
departamentos_sample = departamentos.sample(frac=1.0, random_state = 42)

X = departamentos_sample.drop('valor_m2', axis=1)

y = departamentos_sample[['valor_m2']]

## Sin PolynomialFeatures

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=53)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
se = StandardScaler()
X_train = se.fit_transform(X_train)
X_test = se.fit_transform(X_test)

In [ ]:
alphas = [0.1, 0.125, 0.15, 0.2, 0.25, 0.3, 0.4]

alpha_elasticnet =[]
mean_elasticnet = []
std_elasticnet =[]
predict_elasticnet = []

def f_elasticnet(alpha, X_train, X_test, y_train, y_test):
    model_elasticnet = ElasticNet(alpha=alpha, normalize=False)
    results_elasticnet = cross_val_score(model_elasticnet, X_train, y_train, cv=5, scoring='r2')
    alpha_elasticnet.append(alpha)
    mean_elasticnet.append(np.mean(results_elasticnet))
    std_elasticnet.append(np.std(results_elasticnet))
    model_elasticnet.fit(X_train,y_train)
    predict_elasticnet.append(model_elasticnet.predict(X_test))

for alpha in alphas:
    f_elasticnet(alpha, X_train, X_test, y_train, y_test)

plt.plot(alpha_elasticnet, mean_elasticnet, label='ElasticNet')
plt.legend(loc='lower left')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.tight_layout()
plt.show()

In [ ]:
# Plot the model - predict_ridge - alpha 0.15

dfPredict_elasticnet = pd.DataFrame({'predict_ridge':predict_elasticnet[2]})

plt.plot(y,y, '-.',c='grey')
plt.scatter(dfPredict_elasticnet, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from all values")
plt.ylabel("Actual Values y_test")
plt.show()
print(r2_score(y_test, dfPredict_elasticnet))
print ("MSE:", mean_squared_error(y_test, dfPredict_elasticnet))

## Con PolynomialFeatures

In [ ]:
p = PolynomialFeatures(degree=2).fit(X)
X_features = pd.DataFrame(p.transform(X), columns=p.get_feature_names(X.columns))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=53)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

se = StandardScaler()
X_train = se.fit_transform(X_train)
X_test = se.fit_transform(X_test)

In [ ]:
alphas = [1.5, 2, 2.5, 3, 3.5]

alpha_elasticnet =[]
mean_elasticnet = []
std_elasticnet =[]
predict_elasticnet = []

def f_elasticnet(alpha, X_train, X_test, y_train, y_test):
    model_elasticnet = ElasticNet(alpha=alpha, max_iter=10000, normalize=False)
    results_elasticnet = cross_val_score(model_elasticnet, X_train, y_train, cv=5, scoring='r2')
    alpha_elasticnet.append(alpha)
    mean_elasticnet.append(np.mean(results_elasticnet))
    std_elasticnet.append(np.std(results_elasticnet))
    model_elasticnet.fit(X_train,y_train)
    predict_elasticnet.append(model_elasticnet.predict(X_test))

for alpha in alphas:
    f_elasticnet(alpha, X_train, X_test, y_train, y_test)

plt.plot(alpha_elasticnet, mean_elasticnet, label='ElasticNet')
plt.legend(loc='lower left')
plt.xlabel('alpha')
plt.ylabel('cross validation score')
plt.tight_layout()
plt.show()

In [ ]:
# Plot the model - predict_ridge - alpha 3

dfPredict_elasticnet = pd.DataFrame({'predict_ridge':predict_elasticnet[3]})

plt.plot(y,y, '-.',c='grey')
plt.scatter(dfPredict_elasticnet, y_test, s=30, c='r', marker='+', zorder=10)
plt.xlabel("Predicted Values from all values")
plt.ylabel("Actual Values y_test")
plt.show()
print(r2_score(y_test, dfPredict_elasticnet))
print ("MSE:", mean_squared_error(y_test, dfPredict_elasticnet))